In [333]:
from dateutil.relativedelta import relativedelta
from argon2 import PasswordHasher
from datetime import datetime
from faker import Faker
import pandas as pd
import random as rd
import re

SEED = 29
Faker.seed(SEED)
rd.seed(SEED)

ph = PasswordHasher()
fk = Faker('pt-BR')
QNT = 10

In [334]:
def generate_addresses():
    complements = ['Casa 2', 'Apto. 162', 'Casa B', 'Galpão A', None, None, None, None, None, None]
    raw_addresses = []

    for ad in [fk.address() for _ in range(2 * QNT - 1)]:
        split_ad = ad.split('\n')
        street_address = split_ad[0].split(',')
        street = street_address[0]
        number = int(street_address[1]) if len(street_address) > 1 else 0
        temp, state = split_ad[2].split('/')
        postal_code, city = temp.split(' ', maxsplit=1)
        postal_code = postal_code.replace('-', '')
        complement = rd.choice(complements)

        raw_addresses.append(('Brasil', state, city, street, number, postal_code, complement))

    return raw_addresses

In [335]:
addresses = pd.DataFrame(
    data=generate_addresses(),
    columns=['country', 'state', 'city', 'street', 'number', 'postal_code', 'complement']
)

display(addresses)

,country,state,city,street,number,postal_code,complement
0,Brasil,MT,Monteiro,Trevo de da Paz,9,66001735,None
1,Brasil,SE,Jesus,Estrada de Machado,73,98386448,Apto. 162
2,Brasil,RS,Leão,Lagoa Sá,89,52425186,None
3,Brasil,AL,Silveira,Vale Leonardo Duarte,14,66826240,None
4,Brasil,SC,Alves,Vereda Moreira,0,08022753,None
5,Brasil,AM,Pinto,Alameda José Correia,3,63338353,None
6,Brasil,RO,Monteiro do Sul,Viela Leão,6,92461606,Apto. 162
7,Brasil,PE,Cavalcanti de Ramos,Distrito Cavalcante,89,53707393,None
8,Brasil,RN,Sampaio,Área Porto,90,40143531,None
9,Brasil,MA,Gonçalves dos Dourados,Colônia da Cunha,15,33634485,None


In [360]:
# FIXME: change line 15 from addresses.size to addresses.shape[0]
# FIXME: simplify apply() in line 19

gen_date = lambda: fk.date_between(datetime.now() - relativedelta(years=50), datetime.now() - relativedelta(years=12))
gen_password = lambda size: fk.password(length=size)

users = pd.DataFrame({
    'cpf': [fk.cpf() for _ in range(QNT)],
    'profile_pic': [None] * QNT,
    'email': [fk.email() for _ in range(QNT)],
    'username': [fk.name() for _ in range(QNT)],
    'phone': [fk.phone_number() for _ in range(QNT)],
    'birthday': [gen_date() for _ in range(QNT)],
    'is_active': [True] * QNT,
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [(i % addresses.size + 1) for i in range(QNT)],
})

users['password'] = users['raw_password'].apply(lambda x: ph.hash(x))
users['phone'] = users['phone'].apply(lambda x: re.sub(r'[^0-9]', '', x))
users.to_csv('csv/test_logins.csv', columns=['email', 'raw_password'])
users.drop(columns=['raw_password'], inplace=True)

display(users)

,cpf,profile_pic,email,username,phone,birthday,is_active,address_id,password
0,563.021.478-08,None,sampaiobryan@example.net,Igor Moura,3163130881,1992-12-24,True,1,"$argon2id$v=19$m=65536,t=3,p=4$3gVLiiZJ+uQJ5ql..."
1,187.926.403-03,None,hda-cruz@example.com,Sra. Larissa Pimenta,1169051379,2008-02-05,True,2,"$argon2id$v=19$m=65536,t=3,p=4$fiAh7qdmcVghg70..."
2,184.750.362-44,None,uda-rocha@example.com,Kevin Brito,5508443469294,2013-01-22,True,3,"$argon2id$v=19$m=65536,t=3,p=4$fDKrF6MMrAW+/VX..."
3,074.538.129-41,None,benicio83@example.com,Valentina Ribeiro,01144729996,2013-03-17,True,4,"$argon2id$v=19$m=65536,t=3,p=4$J5vt4VVfciZcJm0..."
4,689.071.234-03,None,barbarafonseca@example.com,Melissa Camargo,8434245687,2004-03-14,True,5,"$argon2id$v=19$m=65536,t=3,p=4$IyVtQCS5C/oSiAn..."
5,930.851.264-06,None,asafemoura@example.com,Oliver da Luz,557154105514,1982-04-11,True,6,"$argon2id$v=19$m=65536,t=3,p=4$hPDROHlxkeI/uS3..."
6,476.039.185-10,None,pachecomariane@example.org,Sr. Felipe Guerra,1162287899,1997-11-24,True,7,"$argon2id$v=19$m=65536,t=3,p=4$J2NeD8nlBUs6wAx..."
7,478.213.650-17,None,claracampos@example.net,Danilo Moraes,5501193396090,1997-05-05,True,8,"$argon2id$v=19$m=65536,t=3,p=4$YO7j0X2FAzXWuaY..."
8,169.075.382-02,None,da-costamariana@example.com,Agatha da Mata,8425344700,1993-01-05,True,9,"$argon2id$v=19$m=65536,t=3,p=4$DkR1V9O3WYXfMhA..."
9,428.917.350-05,None,icostela@example.com,Joaquim Nogueira,1154811587,1977-03-26,True,10,"$argon2id$v=19$m=65536,t=3,p=4$dnPbmE+T4azXL43..."


In [359]:
stores = pd.DataFrame({
    'cnpj': [fk.cnpj() for _ in range(QNT)],
    'name': [fk.company() for _ in range(QNT)],
    'picture_url': [None] * QNT,
    'email': [fk.company_email() for _ in range(QNT)],
    'is_active': [True] * QNT,
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [((i + users.shape[0]) % addresses.shape[0] + 1) for i in range(QNT)]
})

stores['password'] = stores['raw_password'].apply(ph.hash)
stores.to_csv('csv/test_store_logins.csv', columns=['email', 'raw_password'])
stores.drop(columns=['raw_password'], inplace=True)

display(stores)

,cnpj,name,picture_url,email,is_active,address_id,password
0,86.375.249/0001-05,Porto,None,henriqueoliveira@sousa.org,True,11,"$argon2id$v=19$m=65536,t=3,p=4$PTQb2DAdfIcgLky..."
1,64.179.530/0001-21,Casa Grande,None,vitoria85@araujo.net,True,12,"$argon2id$v=19$m=65536,t=3,p=4$4LtT4jGMFySWJp4..."
2,16.327.905/0001-00,Rios,None,maria-helena36@costela.br,True,13,"$argon2id$v=19$m=65536,t=3,p=4$ajzJdqdh+vjDEqT..."
3,83.657.429/0001-83,da Paz Câmara e Filhos,None,thomasmacedo@freitas.com,True,14,"$argon2id$v=19$m=65536,t=3,p=4$CvA9ye+1DDcYJ59..."
4,63.784.150/0001-53,Campos - EI,None,alicia66@pimenta.br,True,15,"$argon2id$v=19$m=65536,t=3,p=4$BSuzvB/Svdcp/vX..."
5,05.891.267/0001-30,Campos,None,natalia74@da.net,True,16,"$argon2id$v=19$m=65536,t=3,p=4$khfdnHvjswRntBs..."
6,62.094.853/0001-97,Castro,None,pmarques@barros.com,True,17,"$argon2id$v=19$m=65536,t=3,p=4$pY4YLSV/JRSzSik..."
7,02.671.583/0001-08,Moraes,None,wcardoso@da.br,True,18,"$argon2id$v=19$m=65536,t=3,p=4$tpZ3LgJU6Fi+8HM..."
8,71.382.945/0001-63,Cunha,None,mendesotto@sousa.com,True,19,"$argon2id$v=19$m=65536,t=3,p=4$906Vw2b5dcgvG6H..."
9,04.561.879/0001-00,Camargo Correia S/A,None,joao-gabriel49@porto.com,True,1,"$argon2id$v=19$m=65536,t=3,p=4$6yP8a72OYAKomV1..."
